In [ ]:
# !pip install opencv-python
# !pip install mediapipe
# !pip install pyautogui
# !pip install pynput

     ---------------------------------------- 0.0/91.7 kB ? eta -:--:--
     -------------------------- ------------- 61.4/91.7 kB 1.7 MB/s eta 0:00:01
     ---------------------------------------- 91.7/91.7 kB 1.3 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: C:\Users\DSG\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [14]:
import cv2
import mediapipe as mp
import util
import pyautogui
from pynput.mouse import Button, Controller
import random

mouse=Controller()

In [3]:
screen_width, screen_height = pyautogui.size()
mpHands= mp.solutions.hands

hands= mpHands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
    model_complexity=1
)

In [16]:
def move_mouse(index_finger_tip):
    if index_finger_tip is not None:
        x=int(index_finger_tip[0]*screen_width)
        y=int(index_finger_tip[1]*screen_height)
        pyautogui.moveTo(x,y)

def find_finger_tip(result):
    if result.multi_hand_landmarks:
        hand_landmarks=result.multi_hand_landmarks[0]
        return hand_landmarks.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP]

    return None

def is_left_click(landmark_list,thumb_index_dist):
    return (util.get_angle(landmark_list[5],landmark_list[6],landmark_list[8])<50 and 
            util.get_distance([landmark_list[9],landmark_list[10],landmark_list[12]])>90 and   
            thumb_index_dist>50)
        
def is_right_click(landmark_list,thumb_index_dist):
    return (util.get_angle(landmark_list[9],landmark_list[10],landmark_list[12])<50 and 
            util.get_distance([landmark_list[5],landmark_list[6],landmark_list[8]])>90 and   
            thumb_index_dist>50)

def is_double_click(landmark_list,thumb_index_dist):
    return (util.get_angle(landmark_list[5],landmark_list[6],landmark_list[8])<50 and 
            util.get_distance([landmark_list[9],landmark_list[10],landmark_list[12]])<50 and   
            thumb_index_dist>50)

def is_screenshot(landmark_list,thumb_index_dist):
    return (util.get_angle(landmark_list[5],landmark_list[6],landmark_list[8])<50 and 
            util.get_distance([landmark_list[9],landmark_list[10],landmark_list[12]])<50 and   
            thumb_index_dist<50)


def detect_gestures(frame,landmark_list,result):
    if len(landmark_list)>=21:
        index_finger_tip=landmark_list[8]  
        thumb_index_distance=util.get_distance([landmark_list[4],landmark_list[5]]) 

        if thumb_index_distance<50 and util.get_angle(landmark_list[5],landmark_list[6],landmark_list[8])>90:
            move_mouse(index_finger_tip)

        elif is_left_click(landmark_list,thumb_index_distance):
            mouse.click(Button.left)
            mouse.release(Button.left)
            cv2.putText(frame, "Left Click", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
            
        elif is_right_click(landmark_list,thumb_index_distance):
            mouse.click(Button.right)
            mouse.release(Button.right)
            cv2.putText(frame, "Right Click", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

        elif is_double_click(landmark_list,thumb_index_distance):
            pyautogui.doubleClick()
            cv2.putText(frame, "Double Click", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
       
        elif is_screenshot(landmark_list,thumb_index_distance):
            im1=pyautogui.screenshot()
            label=random.randint(0,1000)
            im1.save("screenshot_{label}.png")
            cv2.putText(frame, "Screenshot", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
            


In [17]:


def main():
    cap = cv2.VideoCapture(0)
    draw= mp.solutions.drawing_utils
    try:
        while cap.isOpened():
            ret,frame = cap.read()

            if not ret:
                break
            frame = cv2.flip(frame,1)
            frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            result = hands.process(frameRGB)
            landmarks_list=[]

            if result.multi_hand_landmarks:
                hand_landmarks = result.multi_hand_landmarks[0]
                draw.draw_landmarks(frame, hand_landmarks, mpHands.HAND_CONNECTIONS)
                
                for lm in hand_landmarks.landmark:
                    landmarks_list.append((lm.x,lm.y))

            detect_gestures(frame,landmarks_list,result)  

            print(landmarks_list)
            
            cv2.imshow('frame',frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    finally:
        cap.release()
        cv2.destroyAllWindows()

if __name__ == '__main__':
    main()
    

[(0.27245980501174927, 0.835655152797699), (0.26547718048095703, 0.764194667339325), (0.2846716046333313, 0.7031114101409912), (0.3179354667663574, 0.6691190004348755), (0.3531780242919922, 0.6556435823440552), (0.29186874628067017, 0.6740176677703857), (0.341512531042099, 0.6187329292297363), (0.37935325503349304, 0.5941137075424194), (0.4064629077911377, 0.5858544707298279), (0.31579211354255676, 0.6864748001098633), (0.3678482174873352, 0.6225024461746216), (0.4051154851913452, 0.6017500758171082), (0.4314545691013336, 0.5966243147850037), (0.34309062361717224, 0.7109953165054321), (0.3921169936656952, 0.6474443674087524), (0.4242716133594513, 0.6243903636932373), (0.44699394702911377, 0.6150772571563721), (0.3732743561267853, 0.7450978755950928), (0.41540464758872986, 0.6939417123794556), (0.4422288239002228, 0.6679955720901489), (0.46327826380729675, 0.6496662497520447)]
[]
[]
[(0.35636427998542786, 0.6034696102142334), (0.3495844900608063, 0.5465552806854248), (0.3706206083297729